# Data Preprocessing

This notebook contains code to preprocess GDP and Stock Market data for CSE 512 Model vs Modalities project.

## Part I (Data Aggregation)

In [25]:
import numpy as np
import pandas as pd

import os

In this section, we will aggregate data from multiple files and different file types, into a single CSV with required variables and timestamps.

In [26]:
raw_data_path = "../Raw_Data/"
processed_data_path = "../Processed_Data/"

### Stock Data

Read data.

In [27]:
final_df = pd.DataFrame()

for file in ['DIA', 'EEM', 'QQQ', 'SPY', 'VXX']:
    with open(os.path.join(raw_data_path, file) + '.txt') as f:
        data = f.readlines()
        data = [line.strip().split(",") for line in data]
        data = pd.DataFrame(data, columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
        data = data[['datetime', 'close']]
        data['close'] = data['close'].astype(float)
        data.columns = ['datetime', file]
        
        if final_df.shape[0] == 0:
            final_df = data
        else:
            final_df = pd.merge(
                final_df,
                data,
                how='left',
                on='datetime'
            )
            
final_df['datetime'] = pd.to_datetime(final_df['datetime'])

Interpolate missing data.

In [28]:
time_frame = pd.DataFrame(pd.date_range(start=final_df['datetime'].min(), end=final_df['datetime'].max(), freq='1min'))
time_frame.columns = ['datetime']

In [29]:
time_frame = time_frame[time_frame['datetime'].astype(str).str.split(' ').apply(lambda x: x[1] <= '16:00:00')]
time_frame = time_frame[time_frame['datetime'].astype(str).str.split(' ').apply(lambda x: x[1] >= '09:30:00')]

In [30]:
final_df = pd.merge(
    time_frame,
    final_df,
    how='left',
    on='datetime'
)

In [31]:
final_df['date'] = final_df['datetime'].dt.date
temp_df = final_df.fillna(-1).groupby('date')['DIA'].sum()
final_df = final_df[~final_df['date'].isin(temp_df[temp_df == -391].index)]

In [32]:
# Choosing subset of data where we have minimal missing values
final_df = final_df.iloc[40273:]
final_df = final_df.drop(columns='date')

In [33]:
final_df = final_df.fillna(method='ffill')

In [34]:
final_df = final_df.set_index('datetime')
final_df.to_csv(os.path.join(processed_data_path, 'etf_data.csv'))

### GDP Data

In [35]:
gdp_data = pd.read_csv(raw_data_path + 'quarterly_GDP.csv')

In [36]:
select_columns = ['DATE', 'Tbill', 'PPINSA', 'CPI', 'M1NSA', 'Unemp', 'IndProd', 'RGDP']

gdp_data = gdp_data[select_columns]
gdp_data = gdp_data.set_index('DATE')

In [37]:
gdp_data.to_csv(os.path.join(processed_data_path, 'gdp_data.csv'))

## Part 2 (Preprocessing Tests)

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.vector_ar.vecm import coint_johansen

import utilities as util

In [39]:
raw_data_path = "../Raw_Data/"
processed_data_path = "../Processed_Data/"

### Stock Data

In [40]:
stocks_data = pd.read_csv(os.path.join(processed_data_path, "etf_data.csv"), index_col='datetime')
stocks_data = stocks_data.iloc[:10000]

In [41]:
for name, column in stocks_data.iteritems():
    util.adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "DIA" 
 -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -2.1598
 No. Lags Chosen       = 1
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.2212. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "EEM" 
 -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -1.8445
 No. Lags Chosen       = 2
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.3586. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "QQQ" 
 -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stat

In [42]:
stocks_data_diff_1 = util.data_differencing(stocks_data)

In [43]:
for name, column in stocks_data_diff_1.iteritems():
    util.adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "DIA" 
 -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -2726.1364
 No. Lags Chosen       = 0
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "EEM" 
 -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -101.7824
 No. Lags Chosen       = 1
 Critical value 1%     = -3.431
 Critical value 5%     = -2.862
 Critical value 10%    = -2.567
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "QQQ" 
 -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test S

In [44]:
cit = util.cointegration_test(stocks_data_diff_1)

Name   ::  Test Stat > C(95%)    =>   Signif  
 ----------------------------------------
DIA    ::  9347.42   > 60.0627   =>   True
EEM    ::  6571.19   > 40.1749   =>   True
QQQ    ::  4644.69   > 24.2761   =>   True
SPY    ::  3042.08   > 12.3212   =>   True
VXX    ::  1511.19   > 4.1296    =>   True


In [45]:
stocks_data_diff_1.to_csv(os.path.join(processed_data_path, 'etf_data_diff_1.csv'))

### GDP Data

In [47]:
gdp_data = pd.read_csv(os.path.join(processed_data_path, "gdp_data.csv"), index_col='DATE')

In [48]:
for name, column in gdp_data.iteritems():
    util.adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "Tbill" 
 -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -1.613
 No. Lags Chosen       = 7
 Critical value 1%     = -3.463
 Critical value 5%     = -2.876
 Critical value 10%    = -2.574
 => P-Value = 0.4764. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "PPINSA" 
 -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = 0.9446
 No. Lags Chosen       = 12
 Critical value 1%     = -3.464
 Critical value 5%     = -2.876
 Critical value 10%    = -2.575
 => P-Value = 0.9936. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "CPI" 
 -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-

In [51]:
gdp_data_diff_1 = util.data_differencing(gdp_data)

In [53]:
for name, column in gdp_data_diff_1.iteritems():
    util.adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "Tbill" 
 -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -4.8665
 No. Lags Chosen       = 14
 Critical value 1%     = -3.464
 Critical value 5%     = -2.876
 Critical value 10%    = -2.575
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "PPINSA" 
 -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -3.7818
 No. Lags Chosen       = 11
 Critical value 1%     = -3.463
 Critical value 5%     = -2.876
 Critical value 10%    = -2.575
 => P-Value = 0.0031. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "CPI" 
 -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 T

In [54]:
gdp_data_diff_2 = util.data_differencing(gdp_data_diff_1)

In [55]:
for name, column in gdp_data_diff_2.iteritems():
    util.adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "Tbill" 
 -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -6.5132
 No. Lags Chosen       = 12
 Critical value 1%     = -3.464
 Critical value 5%     = -2.876
 Critical value 10%    = -2.575
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "PPINSA" 
 -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -7.9661
 No. Lags Chosen       = 11
 Critical value 1%     = -3.463
 Critical value 5%     = -2.876
 Critical value 10%    = -2.575
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "CPI" 
 -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test

In [57]:
gdp_data_diff_2.to_csv(os.path.join(processed_data_path, 'gdp_data_diff_2.csv'), index=True)